In this notebook we explore two ways of writing model in keras, first one is Andre NG's way of specifying set of equations and second one is adding layers one by one. I personally find first one more intuitive. 

In [1]:
import sklearn

In [2]:
import tensorflow  as tf

In [3]:
import seaborn as sns
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegressionCV

from keras.models import Sequential, Model
from keras.layers import Input, Dense, Activation
from keras.utils import np_utils

Using TensorFlow backend.


In [4]:
iris = sns.load_dataset("iris")
iris.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


In [5]:
X = iris.values[:, :4]
y = iris.values[:, 4]

In [6]:
train_X, test_X, train_y, test_y = train_test_split(X, y, train_size=0.5, test_size=0.5, random_state=0)

In [7]:
lr = LogisticRegressionCV()
lr.fit(train_X, train_y)

LogisticRegressionCV(Cs=10, class_weight=None, cv=None, dual=False,
           fit_intercept=True, intercept_scaling=1.0, max_iter=100,
           multi_class='ovr', n_jobs=1, penalty='l2', random_state=None,
           refit=True, scoring=None, solver='lbfgs', tol=0.0001, verbose=0)

In [8]:
print("Accuracy = {:.2f}".format(lr.score(test_X, test_y)))

Accuracy = 0.83


In [9]:
def one_hot_encode_object_array(arr):
    '''One hot encode a numpy array of objects (e.g. strings)'''
    uniques, ids = np.unique(arr, return_inverse=True)
    return np_utils.to_categorical(ids, len(uniques))

train_y_ohe = one_hot_encode_object_array(train_y)
test_y_ohe = one_hot_encode_object_array(test_y)

In [23]:
def model(input_shape):
    X_input = Input(input_shape)
    
    X = Dense(16)(X_input)
    X = Activation('sigmoid')(X)
    
    X = Dense(3)(X)
    X = Activation('softmax')(X)
    
    model = Model(inputs = X_input, outputs = X, name='firstModel')
    
    return model

mm = model(train_X.shape[1:])
mm.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

mm.fit(train_X, train_y_ohe, epochs=100, batch_size=1, verbose=0)

y_pred = mm.predict(test_X)

print "\nTest input\n", test_X[:5]
print "\nPredicted label", np.argmax(y_pred[:5], axis=-1)
print "\nActual one hot encoding\n", test_y_ohe[:5]
print "\n Actual label", test_y[:5]

print "\n\n\n"

loss, accuracy = mm.evaluate(test_X, test_y_ohe, verbose=0)
print("Accuracy = {:.2f}".format(accuracy))

print "\n\n\n"
mm.summary()


Test input
[[5.8 2.8 5.1 2.4]
 [6.0 2.2 4.0 1.0]
 [5.5 4.2 1.4 0.2]
 [7.3 2.9 6.3 1.8]
 [5.0 3.4 1.5 0.2]]

Predicted label [2 1 0 2 0]

Actual one hot encoding
[[0. 0. 1.]
 [0. 1. 0.]
 [1. 0. 0.]
 [0. 0. 1.]
 [1. 0. 0.]]

 Actual label ['virginica' 'versicolor' 'setosa' 'virginica' 'setosa']




Accuracy = 0.99




_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_12 (InputLayer)        (None, 4)                 0         
_________________________________________________________________
dense_27 (Dense)             (None, 16)                80        
_________________________________________________________________
activation_27 (Activation)   (None, 16)                0         
_________________________________________________________________
dense_28 (Dense)             (None, 3)                 51        
_________________________________________________________________
activation_28 (Activa

In [16]:
model = Sequential()

model.add(Dense(16, input_shape=(4,)))
model.add(Activation('sigmoid'))

model.add(Dense(3))
model.add(Activation('softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=["accuracy"])

model.fit(train_X, train_y_ohe, epochs=100, batch_size=1, verbose=0)

loss, accuracy = model.evaluate(test_X, test_y_ohe, verbose=0)
print("Accuracy = {:.2f}".format(accuracy))

print "\n\n\n"
model.summary()

Accuracy = 0.99




_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_13 (Dense)             (None, 16)                80        
_________________________________________________________________
activation_13 (Activation)   (None, 16)                0         
_________________________________________________________________
dense_14 (Dense)             (None, 3)                 51        
_________________________________________________________________
activation_14 (Activation)   (None, 3)                 0         
Total params: 131
Trainable params: 131
Non-trainable params: 0
_________________________________________________________________
